# Testing Custom Timeouts


## Prerequisites

  * Kubernetes cluster with kubectl authorized
  * curl and grpcurl installed

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:8080`

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-ansible" modified.


In [3]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [4]:
VERSION=!cat ../version.txt
VERSION=VERSION[0]
VERSION

'1.10.0-dev'

## Configure Istio

For this example we will create the default istio gateway for seldon which needs to be called `seldon-gateway`. You can supply your own gateway by adding to your SeldonDeployments resources the annotation `seldon.io/istio-gateway` with values the name of your istio gateway.

In [5]:
%%writefile resources/seldon-gateway.yaml
apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: seldon-gateway
  namespace: istio-system
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"

Overwriting resources/seldon-gateway.yaml


In [6]:
!kubectl create -f resources/seldon-gateway.yaml -n istio-system

Error from server (AlreadyExists): error when creating "resources/seldon-gateway.yaml": gateways.networking.istio.io "seldon-gateway" already exists


### Short timeouts deployment file
Below is the outputs of the Seldon config file required to set custom timeouts.

First we'll show how we can set a short lived example by setting the `"seldon.io/rest-timeout":"1000"` annotation with a fake model that delays for 30 secs.

In [28]:
%%writetemplate resources/model_short_timeouts.json
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  annotations:
    seldon.io/rest-timeout: "1000"
    seldon.io/grpc-timeout: "1000"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          name: classifier
    graph:
      children: []
      name: classifier
      type: MODEL
      parameters:
      - name: delaySecs
        type: INT
        value: "30"       
    name: example
    replicas: 1

We can then apply this Seldon Deployment file we defined above in the `seldon` namespace.

In [30]:
!kubectl apply -f resources/model_short_timeouts.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model unchanged


And wait until the resource runs correctly.

In [31]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "seldon-model-example-0-classifier" rollout to finish: 1 old replicas are pending termination...
Waiting for deployment "seldon-model-example-0-classifier" rollout to finish: 1 old replicas are pending termination...
deployment "seldon-model-example-0-classifier" successfully rolled out


In [32]:
!curl -i -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}'    -X POST http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/predictions    -H "Content-Type: application/json"

In [33]:
!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0]]}}' \
         -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict

ERROR:
  Code: DeadlineExceeded
  Message: context deadline exceeded


Delete this graph and recreate one with a longer timeout

In [34]:
!kubectl delete -f resources/model_short_timeouts.json

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


## Seldon Deployment with Long Timeout

Now we can have a look at how we can set a deployment with a longer timeout.

In [35]:
%%writetemplate resources/model_long_timeouts.json
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  annotations:
    seldon.io/rest-timeout: "40000"
    seldon.io/grpc-timeout: "40000"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          name: classifier
    graph:
      children: []
      name: classifier
      type: MODEL
      parameters:
      - name: delaySecs
        type: INT
        value: "30"       
    name: example
    replicas: 1

And we can apply it to deploy the model with long timeouts.

In [36]:
!kubectl apply -f resources/model_long_timeouts.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


In [37]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "seldon-model-example-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-example-0-classifier" successfully rolled out


This next request will work given that the timeout is much longer.

In [38]:
!curl -i -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}'    -X POST http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/predictions    -H "Content-Type: application/json"













{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.10.0-dev"}}}


In [39]:
!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0]]}}' \
         -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict

{
  "meta": {
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.10.0-dev"
    }
  },
  "data": {
    "names": [
      "proba"
    ],
    "ndarray": [
        [
              0.43782349911420193
            ]
      ]
  }
}


In [69]:
!kubectl delete -f resources/model_long_timeouts.json

seldondeployment.machinelearning.seldon.io "seldon-model" deleted
